In [1]:
import re
from string import punctuation
import pandas as pd
from pprint import pprint
import re
import os
import datetime
import uuid
import jinja2
import pathlib
import docx
from string import punctuation
import librosa

In [2]:
def tokenize(txt):
    lst = re.split(r'([.!?…:]+|$)', txt)
    lst = [ ''.join(x).strip() for x in zip(lst[0::2], lst[1::2]) ]
    while '' in lst:
        lst.remove('')
    return lst

In [3]:
def most(lst):
    if sum(lst) / len(lst) > 0.5:
        return True
    return False

def count_ratios(sents, pattern):
    ratios = []
    for sent in sents:
        sent = sent.lower()
        for punc in punctuation + '1234567890':
            sent = sent.replace(punc, '')
        sent = sent.replace(' ', '')
        if len(sent) == 0:
            continue
        else:
            ratios.append(
                len(pattern.findall(sent)) / len(sent)
            )
#     print(ratios)
    return ratios

def split_by_language(string):
    pattern = re.compile(r'[а-яё]')
    
    time = re.search(r'(?:\d\d?:)?\d?\d:\d\d', string)
    
    if time:
        spl = string.split(time.group(0))
        comm = spl[-1]
        string = spl[0]
        time = ' ' + time.group(0)
    else:
        time = ''
        comm = ''
    
    sents = tokenize(string)
#     print(sents)
    if len(sents) == 1:
        return [string + time + f' COMMENT: {comm}']
    
    for i, sent in enumerate(sents):
        prev_ratios = count_ratios(sents[0:i + 1], pattern)
        next_ratios = count_ratios(sents[i + 1:], pattern)
        
        if all([x < 0.5 for x in prev_ratios]) and all([x > 0.6 for x in next_ratios]):
            return [' '.join(sents[0:i + 1]), ' '.join(sents[i + 1:]) + time + f' COMMENT: {comm}']
    return [string + time + f' COMMENT: {comm}']

In [4]:
def split_text_by_language(text):
    lines = []

    for line in text:
        line = split_by_language(line)
        lines.append(line)
    
    return lines

In [5]:
def parse_lines(lines):

    pattern = re.compile(r'(?:\d\d?:)?\d?\d:\d\d')

    df = []

    for group in lines:
        start = 0
        mini_df = []
        for i, line in enumerate(group):
            data = {}
            if 'COMMENT:' in line:
                splitted = line.split('COMMENT: ')
                line = splitted[0].strip()
                group[i] = line
                comm = splitted[1]
                if comm.strip():
                    data['comment'] = comm.strip()
            time = pattern.search(line)
            if time:
                data['time'] = time.group(0)
                split = pattern.split(line)
                if len(''.join(split).strip()) > 3:
                    data['text'] = group[start:i] + [split[0].strip()]
                else:
                    data['text'] = group[start:i]
                start = i + 1
                mini_df.append(data)
        if not mini_df:
            mini_df.append({
                'time': 'UNK',
                'text': group
            })
        df.extend(mini_df)
    return df

In [6]:
def parse_problem_parts(df):
    for part in df:
        if len(part['text']) == 1:
            if ',  ' in part['text'][0]:
                part['text'] = re.split(r',  +', part['text'][0].strip())
            else:
                part['text'] = re.split(r'\s{3,}', part['text'][0].strip())
    return df

In [7]:
def time_to_ms(str_time):
    if isinstance(str_time, float):
        return str_time * 1000
    time = str_time.split(':')
    if len(time) == 3:
        return (int(time[2]) + int(time[1]) * 60 + int(time[0]) * 3600) * 1000
    else:
        return (int(time[1]) + int(time[0]) * 60) * 1000

In [71]:
def define_time(df, max_time):
    curr_time_start = 0

    for i, rep in enumerate(df):
        rep['problems'] = []
        if rep['time'] == 'UNK':
            if i == len(df) - 1:
                time = max_time * 1000
            else:
                x = i + 1
                if x >= len(df):
                    time = curr_time_start + (max_time * 1000 - curr_time_start) / x
                else:
                    while (time := df[x]['time']) == 'UNK':
                        x += 1
                        if x >= len(df):
                            time = curr_time_start + (max_time * 1000 - curr_time_start) / x
                            break
                    else:
                        n_unk = x - i

                        time = curr_time_start + (time_to_ms(time) - curr_time_start) / (n_unk + 1)
            rep['problems'].append('TIME')
        else:
            time = time_to_ms(rep['time'])
        rep['time'] = time + 100
        rep['start_time'] = curr_time_start + 500

        if len(rep['text']) < 2:
            rep['problems'].append('ONE PART')
        elif len(rep['text']) > 2:
            rep['problems'].append('MORE THAN TWO PARTS')

        curr_time_start = time
    
    return df

In [72]:
def make_data_for_elan(df, add_time=0):

    transcriptions = []
    translations = []
    comments = []
    problems = []
    times = {}
    
    i = 1
    k = 1
    for chunk in df:
        if not chunk['text']:
            continue
            
        if int(chunk['time']) + add_time in times:
            times[int(chunk['start_time']) + 500 + add_time] = k
            times[int(chunk['time']) + 500 + add_time] = k + 1
        else:
            times[int(chunk['start_time']) + add_time] = k
            times[int(chunk['time']) + add_time] = k + 1

        transcriptions.append({
            'id': i,
            'start_time': k,
            'time': k + 1,
            'text': chunk['text'][0].replace('<', '[').replace('>', ']')
        })
        translations.append({
            'id': i + 1,
            'start_time': k,
            'time': k + 1,
            'text': ' '.join(chunk['text'][1:]).replace('<', '[').replace('>', ']') if len(chunk['text']) > 1 else ''
        })
        comments.append({
            'id': i + 2,
            'start_time': k,
            'time': k + 1,
            'text': chunk.get('comment', '')
        })
        problems.append({
            'id': i + 3,
            'start_time': k,
            'time': k + 1,
            'text': ' and '.join(chunk.get('problems', []))
        })
        i += 4
        k += 2
    return transcriptions, translations, comments, problems, times

In [73]:
def render_jinja_html(fname, **context):
    return jinja2.Environment(
        loader=jinja2.FileSystemLoader('.')
    ).get_template(fname).render(context)


def create_template(filename, transcriptions, translations, comments, problems, times):
    s = render_jinja_html('eaf_template.eaf',
                          filename=filename,
                          cur_time=datetime.datetime.now().isoformat(),
                          transcriptions=transcriptions,
                          translations=translations,
                          comments = comments,
                          problems = problems,
                          times = times.items()
                         )
    rxEmptyLines = re.compile('\n\s*\n', flags=re.DOTALL)
    s = rxEmptyLines.sub('\n', s)
    fnameOut = 'eafs/' + filename[:-4] + '.eaf'
    fOut = open(fnameOut, 'w', encoding='utf-8')
    fOut.write(s)
    fOut.close()

In [74]:
MAPPING = {
    'alut-txn-2008-161.docx': 'alut-2008-txn161-VVV-0125.wav',
    'alut-txn-2008-113-14.docx': 'alut-txn-2008-113-VVV-detstvo-2007.WAV',
    'alut-txn-2008-162.docx': 'alut-2008-txn162-VVV-0127.wav',
    'alut-txn-2009-171.docx': 'alut-2009-txn-171_171R_172R-IOK-1-0001.wav',
    'alut-txn-2009-176.docx': 'alut-2009-txn-175_177-IOK-4-0004.wav',
    'alut-txn-2009-178.docx': 'alut-2009-txn-178_178R-RUR-muxomory-0228.wav',
    'alut-txn-2009-180.docx': 'alut-2009-txn-180-DAM-1-0009.wav',
    'alut-txn-2009-181.docx': 'alut-2009-txn-181-DAM-2-0010.wav',
    'alut-txn-2008-111.docx': 'alut-txn-2008-108_112-VVV-2006.WAV',
    'alut-txn-2008-123.docx': 'alut-txn-2008-123-DAM-1-0106.WAV',
    'alut-txn-2008-126-4.docx': 'alut-txn-2008-126-DAM-4-0109.WAV',
    'alut-txn-2008-127-5.docx': 'alut-txn-2008-127-DAM-5-2009.WAV',
    'alut-txn-2008-128-6.docx': 'alut-txn-2008-128-DAM-6-2010.WAV'
}

In [75]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        if para.text.strip().strip(punctuation):
            fullText.append(para.text.strip())
    return '\n'.join(fullText)

In [76]:
def word2eaf(filename, add_time=0):
    max_time = librosa.get_duration(path='eafs/' + MAPPING[filename])
    text = getText('texts/' + filename)
    lines = split_text_by_language(text.splitlines())
    df = parse_lines(lines)
    df = parse_problem_parts(df)
    df = define_time(df, max_time)
    transcriptions, translations, comments, problems, times = make_data_for_elan(df, add_time)
    
#     return times
    
    audio = MAPPING[filename]
    
    create_template(audio, transcriptions, translations, comments, problems, times)

In [77]:
ok_docs = [
    'alut-txn-2008-161.docx',
    'alut-txn-2009-171.docx',
    'alut-txn-2009-178.docx',
    'alut-txn-2009-180.docx',
    'alut-txn-2009-181.docx',
    'alut-txn-2008-127-5.docx',
    'alut-txn-2008-128-6.docx',
    'alut-txn-2008-162.docx',
    'alut-txn-2008-113-14.docx',
    'alut-txn-2009-176.docx',
    'alut-txn-2008-126-4.docx'
]

In [78]:
word2eaf('alut-txn-2008-126-4.docx', 500)

In [79]:
for doc in ok_docs:
    print(doc)
    word2eaf(doc)

alut-txn-2008-161.docx
alut-txn-2009-171.docx
alut-txn-2009-178.docx
alut-txn-2009-180.docx
alut-txn-2009-181.docx
alut-txn-2008-127-5.docx
alut-txn-2008-128-6.docx
alut-txn-2008-162.docx
alut-txn-2008-113-14.docx
alut-txn-2009-176.docx
alut-txn-2008-126-4.docx


In [80]:
word2eaf('alut-txn-2008-111.docx', (27*60 + 6)*1000)

In [81]:
word2eaf('alut-txn-2008-123.docx', int((47*60 + 40)*1000))